In [50]:
import os
from clickhouse_driver import Client
import logging
from datetime import datetime, timedelta

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
vygruzka_table_name = 'Vygruzka'
fleet_table_name = 'fleet'

# Функция для преобразования названий месяцев на русском в номера
def get_month_number(month_name):
    months = {
        "Январь": 1,
        "Февраль": 2,
        "Март": 3,
        "Апрель": 4,
        "Май": 5,
        "Июнь": 6,
        "Июль": 7,
        "Август": 8,
        "Сентябрь": 9,
        "Октябрь": 10,
        "Ноябрь": 11,
        "Декабрь": 12
    }
    return months.get(month_name, None)

# Функция для получения данных из таблицы fleet с приведением типов
def get_fleet_data(client):
    try:
        fleet_query = f"SELECT month, mi8t_count, mi17_count FROM {database_name}.{fleet_table_name}"
        fleet_results = client.execute(fleet_query)
        fleet_dict = {}
        for row in fleet_results:
            month_name = row[0]
            mi8t_count = row[1]
            mi17_count = row[2]
            month_number = get_month_number(month_name)
            if month_number:
                # Приведение mi8t_count и mi17_count к типу Float64, обработка None
                try:
                    mi8t_count = float(mi8t_count) if mi8t_count is not None else None
                except ValueError:
                    logging.warning(f"mi8t_count не является числом для месяца: {month_name}, значение: {mi8t_count}")
                    mi8t_count = None
                try:
                    mi17_count = float(mi17_count) if mi17_count is not None else None
                except ValueError:
                    logging.warning(f"mi17_count не является числом для месяца: {month_name}, значение: {mi17_count}")
                    mi17_count = None
                fleet_dict[month_number] = {
                    'mi8t_count': mi8t_count,
                    'mi17_count': mi17_count
                }
            else:
                logging.warning(f"Не удалось определить номер месяца для названия: {month_name}")
        logging.info(f"Получено данных из fleet: {len(fleet_dict)} месяцев")
        return fleet_dict
    except Exception as e:
        logging.error(f"Ошибка при получении данных из fleet: {e}")
        raise

# Функция для создания основного OLAP куба
def create_multidimensional_olap_cube(client, fleet_dict):
    # Получение уникальных значений serialno из таблицы Vygruzka
    try:
        serialnos_query = f"""
            SELECT DISTINCT serialno 
            FROM {database_name}.{vygruzka_table_name}
        """
        serialnos = client.execute(serialnos_query)
        serialnos = [row[0] for row in serialnos]
        logging.info(f"Количество загруженных агрегатов: {len(serialnos)}")
    except Exception as e:
        logging.error(f"Ошибка при получении serialno: {e}")
        raise

    # Получение начальной даты из таблицы Agregat
    try:
        unload_date_query = f"SELECT MIN(UnloadDate) FROM {database_name}.Agregat WHERE UnloadDate IS NOT NULL"
        unload_date = client.execute(unload_date_query)[0][0]
        logging.info(f"Начальная дата: {unload_date}")
        if unload_date is None:
            raise ValueError("Не удалось получить начальную дату из таблицы Agregat")
    except Exception as e:
        logging.error(f"Ошибка при получении начальной даты: {e}")
        raise

    # Создание диапазона дат +4000 дней начиная с unload_date
    date_range = [unload_date + timedelta(days=i) for i in range(4001)]
    logging.info(f"Диапазон дат создан: {len(date_range)} дней")

    # Создание таблицы для OLAP куба
    olap_table_name = 'OlapCube_VNV'
    try:
        # Удаление таблицы только если она существует
        client.execute(f"DROP TABLE IF EXISTS {database_name}.{olap_table_name}")
        logging.info(f"Существующая таблица {olap_table_name} удалена (если существовала)")
    except Exception as e:
        logging.error(f"Ошибка при удалении таблицы {olap_table_name}: {e}")
        raise

    create_olap_query = f"""
    CREATE TABLE IF NOT EXISTS {database_name}.{olap_table_name} (
        serialno String,
        Dates Date,
        Status_P Nullable(String),
        repair_days Nullable(Float64),
        sne Nullable(Float64),
        ppr Nullable(Float64),
        Status Nullable(String),
        daily_flight_hours Nullable(Float64),
        daily_flight_hours_f Nullable(Float64),
        BR Nullable(Float64),
        ll Nullable(Float64),
        oh Nullable(Float64),
        threshold Nullable(Float64),
        Effectivity Nullable(String),
        mi8t_count Nullable(Float64),
        mi17_count Nullable(Float64),
        balance_mi8t Float64,
        balance_mi17 Float64,
        balance_total Float64,
        stock_mi8t Float64,
        stock_mi17 Float64,
        stock_total Float64,
        location String,         # Добавлено
        ac_typ String            # Добавлено
    ) ENGINE = MergeTree()
    ORDER BY (serialno, Dates)
    """
    try:
        client.execute(create_olap_query)
        logging.info(f"Таблица {olap_table_name} успешно создана")
    except Exception as e:
        logging.error(f"Ошибка при создании таблицы {olap_table_name}: {e}")
        raise

    # Подготовка данных для вставки в OLAP куб
    olap_data = []
    total_records = len(serialnos) * len(date_range)
    logging.info(f"Начинается подготовка данных для вставки: {total_records} записей")

    for idx, serialno in enumerate(serialnos, start=1):
        if idx % 10 == 0:
            logging.info(f"Подготовлено {idx}/{len(serialnos)} serialno")

        for date_idx, date in enumerate(date_range):
            month_number = date.month
            fleet_counts = fleet_dict.get(month_number, {'mi8t_count': None, 'mi17_count': None})

            olap_data.append((
                serialno,
                date,
                None,  # Status_P
                None,  # repair_days
                None,  # sne
                None,  # ppr
                None,  # Status
                None,  # daily_flight_hours
                None,  # daily_flight_hours_f
                None,  # BR
                None,  # ll
                None,  # oh
                None,  # threshold
                None,  # Effectivity
                fleet_counts['mi8t_count'],
                fleet_counts['mi17_count'],
                0.0,  # balance_mi8t
                0.0,  # balance_mi17
                0.0,  # balance_total
                0.0,  # stock_mi8t
                0.0,  # stock_mi17
                0.0,  # stock_total
                '',   # location Добавлено (укажите нужное значение или оставьте пустым)
                ''    # ac_typ Добавлено (укажите нужное значение или оставьте пустым)
            ))

            # Пакетная вставка каждые 100000 записей
            if len(olap_data) >= 100000:
                try:
                    client.execute(f"INSERT INTO {database_name}.{olap_table_name} VALUES", olap_data)
                    logging.info(f"Вставлено {len(olap_data)} записей в {olap_table_name}")
                    olap_data = []
                except Exception as e:
                    logging.error(f"Ошибка при вставке данных в {olap_table_name}: {e}")
                    raise

    # Вставка оставшихся записей
    if olap_data:
        try:
            client.execute(f"INSERT INTO {database_name}.{olap_table_name} VALUES", olap_data)
            logging.info(f"Вставлено {len(olap_data)} записей в {olap_table_name}")
        except Exception as e:
            logging.error(f"Ошибка при вставке оставшихся данных в {olap_table_name}: {e}")
            raise

    logging.info("Все данные успешно загружены в основной OLAP куб")

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name  # Добавлено указание базы данных
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Создание и загрузка OLAP куба
try:
    fleet_dict = get_fleet_data(client)
    create_multidimensional_olap_cube(client, fleet_dict)
    logging.info("Многомерный OLAP куб успешно создан и данные загружены")
except Exception as e:
    logging.error(f"Произошла ошибка при создании OLAP куба: {e}", exc_info=True)


2024-11-26 13:26:37,940 - INFO - Подключение к ClickHouse успешно установлено
2024-11-26 13:26:37,949 - INFO - Получено данных из fleet: 12 месяцев
2024-11-26 13:26:37,954 - INFO - Количество загруженных агрегатов: 185
2024-11-26 13:26:37,960 - INFO - Начальная дата: 2024-11-25
2024-11-26 13:26:37,963 - INFO - Диапазон дат создан: 4001 дней
2024-11-26 13:26:37,967 - INFO - Существующая таблица OlapCube_VNV удалена (если существовала)
2024-11-26 13:26:37,977 - INFO - Таблица OlapCube_VNV успешно создана
2024-11-26 13:26:37,977 - INFO - Начинается подготовка данных для вставки: 740185 записей
2024-11-26 13:26:38,014 - INFO - Подготовлено 10/185 serialno
2024-11-26 13:26:38,062 - INFO - Подготовлено 20/185 serialno
2024-11-26 13:26:38,846 - INFO - Вставлено 100000 записей в OlapCube_VNV
2024-11-26 13:26:38,874 - INFO - Подготовлено 30/185 serialno
2024-11-26 13:26:38,913 - INFO - Подготовлено 40/185 serialno
2024-11-26 13:26:38,964 - INFO - Подготовлено 50/185 serialno
2024-11-26 13:26:39

In [51]:
import os
from clickhouse_driver import Client
import logging
from datetime import datetime, timedelta

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
vygruzka_table_name = 'Vygruzka'

# Функция для получения данных из таблицы Vygruzka для OLAP куба
def get_vygruzka_data(client):
    try:
        vygruzka_query = f"""
            SELECT serialno, ac_typ, BR, ll, oh, threshold, Effectivity
            FROM {database_name}.{vygruzka_table_name}
            WHERE serialno IS NOT NULL
        """
        vygruzka_results = client.execute(vygruzka_query)
        vygruzka_dict = {row[0]: row[1:] for row in vygruzka_results}
        logging.info(f"Получено данных из Vygruzka: {len(vygruzka_dict)} записей")
        return vygruzka_dict
    except Exception as e:
        logging.error(f"Ошибка при получении данных из Vygruzka: {e}")
        raise

# Функция для дополнения таблицы OLAP новыми полями, если они отсутствуют
def alter_olap_table(client, olap_table_name):
    try:
        alter_query = f"""
        ALTER TABLE {database_name}.{olap_table_name}
        ADD COLUMN IF NOT EXISTS ac_typ String,
        ADD COLUMN IF NOT EXISTS BR Float64,
        ADD COLUMN IF NOT EXISTS ll Float64,
        ADD COLUMN IF NOT EXISTS oh Float64,
        ADD COLUMN IF NOT EXISTS threshold Float64,
        ADD COLUMN IF NOT EXISTS Effectivity String
        """
        client.execute(alter_query)
        logging.info(f"Таблица {olap_table_name} успешно дополнена новыми полями")
    except Exception as e:
        logging.error(f"Ошибка при изменении таблицы {olap_table_name}: {e}")
        raise

# Функция для дополнения основного OLAP куба новыми полями
def extend_olap_cube(client, vygruzka_dict):
    # Получение уникальных значений serialno из таблицы Vygruzka
    try:
        serialnos = list(vygruzka_dict.keys())
        logging.info(f"Количество загруженных агрегатов: {len(serialnos)}")
    except Exception as e:
        logging.error(f"Ошибка при получении serialno: {e}")
        raise

    # Получение начальной даты из таблицы Agregat
    try:
        unload_date_query = f"SELECT MIN(UnloadDate) FROM {database_name}.Agregat WHERE UnloadDate IS NOT NULL"
        unload_date = client.execute(unload_date_query)[0][0]
        logging.info(f"Начальная дата: {unload_date}")
        if unload_date is None:
            raise ValueError("Не удалось получить начальную дату из таблицы Agregat")
    except Exception as e:
        logging.error(f"Ошибка при получении начальной даты: {e}")
        raise

    # Создание диапазона дат +4000 дней начиная с unload_date
    date_range = [unload_date + timedelta(days=i) for i in range(4001)]
    logging.info(f"Диапазон дат создан: {len(date_range)} дней")

    # Дополнение таблицы для OLAP куба
    olap_table_name = 'OlapCube_VNV'
    alter_olap_table(client, olap_table_name)

    # Подготовка данных для вставки в OLAP куб
    olap_data = []
    total_records = len(serialnos) * len(date_range)
    logging.info(f"Начинается подготовка данных для вставки: {total_records} записей")

    for idx, serialno in enumerate(serialnos, start=1):
        if idx % 10 == 0:
            logging.info(f"Подготовлено {idx}/{len(serialnos)} serialno")

        ac_typ, BR, ll, oh, threshold, Effectivity = vygruzka_dict[serialno]

        # Приведение типов и обработка значений
        BR = float(BR) if BR is not None else 0.0
        ll = float(ll) if ll is not None else 0.0
        oh = float(oh) if oh is not None else 0.0
        threshold = float(threshold) if threshold is not None else 0.0
        ac_typ = ac_typ if ac_typ is not None else ""
        Effectivity = Effectivity if Effectivity is not None else ""

        for date_idx, date in enumerate(date_range):
            olap_data.append((
                serialno,
                date,
                ac_typ,    # ac_typ
                BR,        # BR
                ll,        # ll
                oh,        # oh
                threshold,  # threshold
                Effectivity # Effectivity
            ))

            # Пакетная вставка каждые 100000 записей
            if len(olap_data) >= 100000:
                try:
                    client.execute(f"INSERT INTO {database_name}.{olap_table_name} (serialno, Dates, ac_typ, BR, ll, oh, threshold, Effectivity) VALUES", olap_data)
                    logging.info(f"Вставлено {len(olap_data)} записей в {olap_table_name}")
                    olap_data = []
                except Exception as e:
                    logging.error(f"Ошибка при вставке данных в {olap_table_name}: {e}")
                    raise

    # Вставка оставшихся записей
    if olap_data:
        try:
            client.execute(f"INSERT INTO {database_name}.{olap_table_name} (serialno, Dates, ac_typ, BR, ll, oh, threshold, Effectivity) VALUES", olap_data)
            logging.info(f"Вставлено {len(olap_data)} записей в {olap_table_name}")
        except Exception as e:
            logging.error(f"Ошибка при вставке оставшихся данных в {olap_table_name}: {e}")
            raise

    logging.info("Все данные успешно загружены в OLAP куб")

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name  # Добавлено указание базы данных
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Дополнение OLAP куба новыми полями
try:
    vygruzka_dict = get_vygruzka_data(client)
    extend_olap_cube(client, vygruzka_dict)
    logging.info("Многомерный OLAP куб успешно дополнен новыми данными")
except Exception as e:
    logging.error(f"Произошла ошибка при дополнении OLAP куба: {e}", exc_info=True)


2024-11-26 13:26:46,902 - INFO - Подключение к ClickHouse успешно установлено
2024-11-26 13:26:46,912 - INFO - Получено данных из Vygruzka: 185 записей
2024-11-26 13:26:46,913 - INFO - Количество загруженных агрегатов: 185
2024-11-26 13:26:46,919 - INFO - Начальная дата: 2024-11-25
2024-11-26 13:26:46,923 - INFO - Диапазон дат создан: 4001 дней
2024-11-26 13:26:46,930 - INFO - Таблица OlapCube_VNV успешно дополнена новыми полями
2024-11-26 13:26:46,932 - INFO - Начинается подготовка данных для вставки: 740185 записей
2024-11-26 13:26:46,948 - INFO - Подготовлено 10/185 serialno
2024-11-26 13:26:46,965 - INFO - Подготовлено 20/185 serialno
2024-11-26 13:26:47,327 - INFO - Вставлено 100000 записей в OlapCube_VNV
2024-11-26 13:26:47,338 - INFO - Подготовлено 30/185 serialno
2024-11-26 13:26:47,357 - INFO - Подготовлено 40/185 serialno
2024-11-26 13:26:47,373 - INFO - Подготовлено 50/185 serialno
2024-11-26 13:26:47,709 - INFO - Вставлено 100000 записей в OlapCube_VNV
2024-11-26 13:26:47,7

In [52]:
import os
from clickhouse_driver import Client
import logging

# Настройки логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Настройки подключения к ClickHouse
clickhouse_host = '10.95.19.132'
clickhouse_port = 9000
clickhouse_user = 'default'
# Рекомендуется хранить пароль в переменных окружения для безопасности
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = 'default'
vygruzka_table_name = 'Vygruzka'

# Функция для проверки существования колонки в таблице
def check_column_exists(client, table_name, column_name):
    try:
        query = f"""
            SELECT name FROM system.columns
            WHERE database = '{database_name}' AND table = '{table_name}' AND name = '{column_name}'
        """
        result = client.execute(query)
        return len(result) > 0
    except Exception as e:
        logging.error(f"Ошибка при проверке существования колонки {column_name} в таблице {table_name}: {e}")
        raise

# Функция для обновления данных в таблице OLAP на основе данных из Vygruzka с фильтрацией по location, содержащему "RA-"
def update_location_data(client):
    olap_table_name = 'OlapCube_VNV'
    column_name = 'location'

    # Проверка существования колонки location в таблице OlapCube_VNV
    if not check_column_exists(client, olap_table_name, column_name):
        logging.error(f"Колонка {column_name} не существует в таблице {olap_table_name}")
        return

    # Получение данных из таблицы Vygruzka с фильтрацией по location, содержащему "RA-"
    try:
        vygruzka_query = f"""
            SELECT serialno, location
            FROM {database_name}.{vygruzka_table_name}
            WHERE serialno IS NOT NULL AND location LIKE 'RA-%'
        """
        vygruzka_results = client.execute(vygruzka_query)
        logging.info(f"Получено данных из Vygruzka: {len(vygruzka_results)} записей, содержащих 'RA-' в поле location")
    except Exception as e:
        logging.error(f"Ошибка при получении данных из Vygruzka: {e}")
        raise

    # Обновление таблицы OLAP на основе данных из Vygruzka
    try:
        for serialno, location in vygruzka_results:
            update_query = f"""
                ALTER TABLE {database_name}.{olap_table_name} UPDATE location = '{location}'
                WHERE serialno = '{serialno}'
            """
            client.execute(update_query)
        logging.info(f"Данные о местоположении успешно обновлены в {olap_table_name}")
    except Exception as e:
        logging.error(f"Ошибка при обновлении данных в {olap_table_name}: {e}")
        raise

# Подключение к ClickHouse с указанием базы данных
try:
    client = Client(
        host=clickhouse_host,
        port=clickhouse_port,
        user=clickhouse_user,
        password=clickhouse_password,
        database=database_name
    )
    logging.info("Подключение к ClickHouse успешно установлено")
except Exception as e:
    logging.error(f"Не удалось подключиться к ClickHouse: {e}")
    raise

# Обновление данных о местоположении в OLAP кубе
try:
    update_location_data(client)
    logging.info("Данные о местоположении успешно обновлены в OLAP кубе")
except Exception as e:
    logging.error(f"Произошла ошибка при обновлении данных о местоположении: {e}", exc_info=True)


2024-11-26 13:26:52,525 - INFO - Подключение к ClickHouse успешно установлено
2024-11-26 13:26:52,537 - INFO - Получено данных из Vygruzka: 162 записей, содержащих 'RA-' в поле location
2024-11-26 13:26:53,239 - INFO - Данные о местоположении успешно обновлены в OlapCube_VNV
2024-11-26 13:26:53,241 - INFO - Данные о местоположении успешно обновлены в OLAP кубе
